# Práctica 4 - Filtrado de Spam usando Bayes Ingenuo

Guarda una copia de este cuaderno en tu Google Drive para poder editarla y ejecutarla.

El propio cuaderno será tu informe de la práctica. Puedes añadir tantas secciones de código y de texto como consideres necesario para resolver todos los ejercicios propuestos y analizar los resultados obtenidos. Una vez hayas terminado, descarga el notebook en formato ipynb y súbelo a Moodle en la tarea habilitada para la P4 con el nombre NIP_P4.ipynb

Imports necesarios para ejecutar la práctica

In [1]:
import numpy as np
import json
import glob
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.model_selection import KFold

Carga del fichero ZIP con todos los correos

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1hYha8kSpbAhGIHAfygLFmGHIzHGAK5f8' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1hYha8kSpbAhGIHAfygLFmGHIzHGAK5f8" -O "enron.zip" && rm -rf /tmp/cookies.txt
!unzip "enron.zip"

Lectura de los emails y carga en las estructuras de datos

In [30]:
def read_folder(folder):
    mails = []
    file_list = glob.glob(folder)  # List mails in folder
    num_files = len(file_list)
    for i in range(0, num_files):
        i_path = file_list[i]
        i_file = open(i_path, 'rb')
        i_str = i_file.read()
        i_text = i_str.decode('utf-8', errors='ignore')  # Convert to Unicode
        mails.append(i_text)  # Append to the mail structure
        i_file.close()
    return mails

def load_enron_folders(datasets):
    path = './'
    ham = []
    spam = []
    for j in datasets:
        ham  = ham  + read_folder(path + '/enron' + str(j) + '/ham/*.txt')
        spam = spam + read_folder(path + '/enron' + str(j) + '/spam/*.txt')
    num_ham  = len(ham)
    num_spam = len(spam)
    print("mails:", num_ham+num_spam)
    print("ham  :", num_ham)
    print("spam :", num_spam)

    mails = ham + spam
    labels = [0]*num_ham + [1]*num_spam
    mails, labels = shuffle(mails, labels, random_state=0)
    return mails, labels

print("Loading files...")

print("------Loading train and validation data--------")
mails, y = load_enron_folders([1,2,3,4,5])

print("--------------Loading Test data----------------")
mails_test, y_test = load_enron_folders([6])

Loading files...
------Loading train and validation data--------
mails: 27716
ham  : 15045
spam : 12671
--------------Loading Test data----------------
mails: 6000
ham  : 1500
spam : 4500


Código para generar una bolsa de palabras que cuenta el número de apariciones de cada palabra en la lista de correos

Crea una matriz X con tantas filas como correos (27716) y tantas columnas como palabras de la BD. El elemento (i,j) de la matriz contiene el número de ocurrencias de la palabra j en el correo i

In [4]:
vectorizer  = CountVectorizer(ngram_range=(1, 1))  # Instancia de bolsa de palabras con palabras individuales como características
X = vectorizer.fit_transform(mails)                # Generación y cálculo de la bolsa de palabras en base a los datos de entrenamiento
X_test = vectorizer.transform(mails_test)          # Cáclulo de la bolsa de palabras con los datos de test

Aprendizaje de las probabilidades utilizando un modelo de distribución Bernoulli.

Consulta la documentación de sklearn para entender los parámetros.

In [ ]:
classifier = BernoulliNB(alpha=1.0, fit_prior=True, class_prior=None) # Instancia de clasificador de Bayes Ingenuo con distribución de Bernoulli
classifier.fit(X,y) # Cálculo de las probabilidades asociadas a cada palabra de la bolsa

Cálculo de métricas del clasificador utilizando los datos de test

In [ ]:
y_pred = classifier.predict(X_test)
f1_score=metrics.f1_score(y_test, y_pred)
print('%s %2.2f%s' % ('F1-score of the test: ', 100*f1_score, '%' ))
C=metrics.confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(C)
metrics.plot_precision_recall_curve(classifier,X_test,y_test)

## 1. Clasificador Bayes ingénuo

### 1.1. Distribución binomial

In [ ]:
# Chapuza para suprimir avisos de la funcion plot 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Crea y obtiene métricas para un clasificador mediante Bayes ingénuo utliizando
# distribuciones binomiales 
#
# suavizado: Hiperparámetro de suavizado de Laplace
# imprConfusion: Imprime matriz confusión
# imprPR: Imprime gráfica precision-recall
def bayesIngenuoBernoulli(suavizado, imprConfusion = False, imprPR = False):
  # Instancia el clasificador
  clasificador = BernoulliNB(alpha = suavizado)

  # Calcula probabilidades utilizando los datos de entrenamiento
  clasificador.fit(X,y)

  # Realiza clasificación con los datos de prueba
  y_prediccion = clasificador.predict(X_test)
  print("Bayes ingénuo, distribución binomial, suavizado = %2.2f" % (suavizado))

  # Puntuación F1
  f1_score = metrics.f1_score(y_test, y_prediccion)
  print("\nPuntuación F1: %2.2f" % (100 * f1_score))

  # Matriz de confusión
  if imprConfusion:
    confusion = metrics.confusion_matrix(y_test, y_prediccion)
    print("Matriz de confusión:")
    print(confusion)

  # Recta precision-recall
  if imprPR:
    metrics.plot_precision_recall_curve(clasificador, X_test, y_test)

  print("\n")

  return f1_score

bayesIngenuoBernoulli(1.0, True, True)


### 1.2. Distribución multinomial

In [ ]:
# Crea y obtiene métricas para un clasificador mediante Bayes ingénuo utliizando
# distribuciones multinomiales. Imprime métricas y devuelve la puntuación F1
#
# suavizado: Hiperparámetro de suavizado de Laplace
# imprConfusion: Imprime matriz confusión
# imprPR: Imprime gráfica precision-recall
def bayesIngenuoMultinomial(suavizado, imprConfusion = False, imprPR = False):
  # Instancia el clasificador
  clasificador = MultinomialNB(alpha = suavizado)

  # Calcula probabilidades utilizando los datos de entrenamiento
  clasificador.fit(X,y)

  # Realiza clasificación con los datos de prueba
  y_prediccion = clasificador.predict(X_test)
  print("Bayes ingénuo, distribución multinomial, suavizado = %2.2f" % (suavizado))

  # Puntuación F1
  f1_score = metrics.f1_score(y_test, y_prediccion)
  print("\nPuntuación F1: %2.2f" % (100 * f1_score))

  # Matriz de confusión
  if imprConfusion:
    confusion = metrics.confusion_matrix(y_test, y_prediccion)
    print("Matriz de confusión:")
    print(confusion)

  # Recta precision-recall
  if imprPR:
    metrics.plot_precision_recall_curve(clasificador, X_test, y_test)

  print("\n")

  return f1_score


bayesIngenuoMultinomial(1.0, True, True)

## 2. Pruebas según el hiperparámetro de suavizado de Laplace

### 2.1. Distribución binomial

In [ ]:
# Obtiene el mejor suavizado a partir de la puntuación F1
# Bayes ingénuo binomial
def mejorSuavizadoBinomial():
  mejor_f1 = 0.0
  mejor_suavizado = 0.0
  i = 0.1

  while i <= 2.0:
    f1_score = bayesIngenuoBernoulli(i)

    if f1_score > mejor_f1:
      mejor_f1 = f1_score
      mejor_suavizado = i

    i+=0.05

  print("El hiperparámetro %2.2f ha dado el mejor resultado con una puntuación de %2.2f" % (mejor_suavizado, 100*mejor_f1))

  return (mejor_suavizado, 100*mejor_f1)

mejorSuavizadoBinomial()  

### 2.2. Distribución multinomial

In [ ]:
# Obtiene el mejor suavizado a partir de la puntuación F1
# Bayes ingénuo multinomial
def mejorSuavizadoMultinomial():
  mejor_f1 = 0.0
  mejor_suavizado = 0.0
  i = 0.1

  while i <= 2.0:
    f1_score = bayesIngenuoMultinomial(i)

    if (f1_score) > mejor_f1:
      mejor_f1 = f1_score
      mejor_suavizado = i
    
    i+=0.05

  print("El hiperparámetro %2.2f ha dado el mejor resultado con una puntuación de %2.2f" % (mejor_suavizado, 100*mejor_f1))
  return (mejor_suavizado, 100*mejor_f1)


mejorSuavizadoMultinomial()

## 3. Utilización de bigramas

In [ ]:
# Genera y calcula la bolsa de palabras, esta vez pasando de generar
# unigramas a partir de las palabras a bigramas

vectorizer  = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(mails)          
X_test = vectorizer.transform(mails_test)

mejorSuavizadoBinomial()
print("\n")
mejorSuavizadoMultinomial()

## 4. Evaluación del mejor clasificador

In [ ]:
# Prueba distintas combinaciones utilizando solo el F1 para comparar

vectorizer  = CountVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(mails)          
X_test = vectorizer.transform(mails_test)

mejor_binomial_unigrama = mejorSuavizadoBinomial()
print("\n")
mejor_multinomial_unigrama = mejorSuavizadoMultinomial()

vectorizer  = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(mails)          
X_test = vectorizer.transform(mails_test)

mejor_binomial_bigrama = mejorSuavizadoBinomial()
print("\n")
mejor_multinomial_bigrama = mejorSuavizadoMultinomial()
print("\n")

print("De entre los casos analizados, los mejores F1 son:")

print("Unigrama-binomial: Suavizado = %2.2f, F1 = %2.2f" % ((mejor_binomial_unigrama[0], mejor_binomial_unigrama[1])))
print("Unigrama-multinomial: Suavizado = %2.2f, F1 = %2.2f" % ((mejor_multinomial_unigrama[0], mejor_multinomial_unigrama[1])))
print("Bigrama-binomial: Suavizado = %2.2f, F1 = %2.2f" % ((mejor_binomial_bigrama[0], mejor_binomial_bigrama[1])))
print("Bigrama-multinomial: Suavizado = %2.2f, F1 = %2.2f" % ((mejor_multinomial_bigrama[0], mejor_multinomial_bigrama[1])))

Utilizando solo la puntuación F1 para obtener el mejor umbral, se encuentra que el utilizando unigramas el mejor resultado se da con un suavizado mínimo, 0.1, mientras que utilizando bigramas para la bolsa de palabras un buen umbral es 1.95 utilizando una distribución binomial y 0.15 para una distribución binomial. El mejor F1 lo da utilizando bigramas con una distribución multinomial.

In [ ]:
# Obtiene más estadisticas para las mejores puntuaciones

vectorizer  = CountVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(mails)          
X_test = vectorizer.transform(mails_test)

bayesIngenuoBernoulli(0.1, True, True)
bayesIngenuoMultinomial(0.1, True, True)

vectorizer  = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(mails)          
X_test = vectorizer.transform(mails_test)

bayesIngenuoBernoulli(1.95, True, True)
bayesIngenuoMultinomial(0.15, True, True)

A partir de la curva precision-recall, se prueba con distintos umbrales tal que se ajuste mejor a ella. Para ello se obtienen las probabilidades de que los correos sean clasificados correctamente.

In [ ]:
# Crea y obtiene métricas para un clasificador mediante Bayes ingénuo utliizando
# distribuciones binomiales, dado un umbral personalizado. Imprime métricas 
# y devuelve la puntuación F1
#
# suavizado: Hiperparámetro de suavizado de Laplace
# imprConfusion: Imprime matriz confusión
# imprPR: Imprime gráfica precision-recall
def bayesIngenuoBinomialDadoUmbral(suavizado, umbral, imprConfusion = False, imprPR = False):
  # Instancia el clasificador
  clasificador = BernoulliNB(alpha = suavizado)

  # Calcula probabilidades utilizando los datos de entrenamiento
  clasificador.fit(X,y)

  # Realiza clasificación con los datos de prueba, aplicando el umbral
  # a partir de las probabilidades de clasificar correctamente el dato
  y_prediccion = (clasificador.predict_proba(X_test)[:,1] >= umbral).astype(bool)
  print("Bayes ingénuo, distribución binomial, suavizado = %2.2f, umbral = %2.2f" % (suavizado, umbral))

  # Puntuación F1
  f1_score = metrics.f1_score(y_test, y_prediccion)
  print("\nPuntuación F1: %2.2f" % (100 * f1_score))

  # Matriz de confusión
  if imprConfusion:
    confusion = metrics.confusion_matrix(y_test, y_prediccion)
    print("Matriz de confusión:")
    print(confusion)

  # Recta precision-recall
  if imprPR:
    metrics.plot_precision_recall_curve(clasificador, X_test, y_test)

  print("\n")

  return f1_score

# Crea y obtiene métricas para un clasificador mediante Bayes ingénuo utliizando
# distribuciones multinomiales, dado un umbral personalizado. Imprime métricas 
# y devuelve la puntuación F1
#
# suavizado: Hiperparámetro de suavizado de Laplace
# imprConfusion: Imprime matriz confusión
# imprPR: Imprime gráfica precision-recall
def bayesIngenuoMultinomialDadoUmbral(suavizado, umbral, imprConfusion = False, imprPR = False):
  # Instancia el clasificador
  clasificador = MultinomialNB(alpha = suavizado)

  # Calcula probabilidades utilizando los datos de entrenamiento
  clasificador.fit(X,y)

  # Realiza clasificación con los datos de prueba, aplicando el umbral
  # a partir de las probabilidades de clasificar correctamente el dato
  y_prediccion = (clasificador.predict_proba(X_test)[:,1] >= umbral).astype(bool)
  print("Bayes ingénuo, distribución multinomial, suavizado = %2.2f, umbral = %2.2f" % (suavizado, umbral))

  # Puntuación F1
  f1_score = metrics.f1_score(y_test, y_prediccion)
  print("\nPuntuación F1: %2.2f" % (100 * f1_score))

  # Matriz de confusión
  if imprConfusion:
    confusion = metrics.confusion_matrix(y_test, y_prediccion)
    print("Matriz de confusión:")
    print(confusion)

  # Recta precision-recall
  if imprPR:
    metrics.plot_precision_recall_curve(clasificador, X_test, y_test)

  print("\n")

  return f1_score

def mejorUmbralBinomial(suavizado):
  mejor_umbral = 0.0
  mejor_f1 = 0.0
  i = 0.05

  while i <= 1.0:
    f1 = bayesIngenuoBinomialDadoUmbral(suavizado, i, True, False)
    if f1 > mejor_f1:
      mejor_f1 = f1
      mejor_umbral = i
    i += 0.05

  return (mejor_umbral, mejor_f1)

def mejorUmbralMultinomial(suavizado):
  mejor_umbral = 0.0
  mejor_f1 = 0.0
  i = 0.05

  while i <= 1.0:
    f1 = bayesIngenuoMultinomialDadoUmbral(suavizado, i, True, False)
    if f1 > mejor_f1:
      mejor_f1 = f1
      mejor_umbral = i
    i += 0.05

  return (mejor_umbral, mejor_f1)


vectorizer  = CountVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(mails)          
X_test = vectorizer.transform(mails_test)

res1 = mejorUmbralBinomial(0.1)
res2 = mejorUmbralMultinomial(0.1)

vectorizer  = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(mails)          
X_test = vectorizer.transform(mails_test)

res3 = mejorUmbralBinomial(1.95)
res4 = mejorUmbralMultinomial(0.15)

print("Los mejores resultados para cada caso:")
print("Unigrama-binomial: Suavizado = %2.2f, Umbral = %2.2f, F1 = %2.2f" % ((0.1, res1[0], 100*res1[1])))
print("Unigrama-multinomial: Suavizado = %2.2f, Umbral = %2.2f, F1 = %2.2f" % ((0.1, res2[0], 100*res2[1])))
print("Bigrama-binomial: Suavizado = %2.2f, Umbral = %2.2f, F1 = %2.2f" % ((1.95, res3[0], 100*res3[1])))
print("Bigrama-multinomial: Suavizado = %2.2f, Umbral = %2.2f, F1 = %2.2f" % ((0.15, res4[0], 100*res4[1])))

In [ ]:
# Obtiene más estadisticas para las mejores puntuaciones

vectorizer  = CountVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(mails)          
X_test = vectorizer.transform(mails_test)

bayesIngenuoBinomialDadoUmbral(0.1, 0.85, True, True)
bayesIngenuoMultinomialDadoUmbral(0.1, 0.35, True, True)

vectorizer  = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(mails)          
X_test = vectorizer.transform(mails_test)

bayesIngenuoBinomialDadoUmbral(1.95, 0.95, True, True)
bayesIngenuoMultinomialDadoUmbral(0.15, 0.05, True, True)

## Conclusiones

1. La distribución con la mejor puntuación F1 ajustados suavizado y umbral es la distribución multinomial utilizando bigramas, suavizado 0.15 y umbral 0.05.
2. La mejor puntuación F1 no significa que sea la mejor opción. Para el caso anterior, por ejemplo, se puede apreciar como la puntuación queda sesgada porque detecta mejor el correo spam a costa de mas falsos positivos de correos ham. Conforme aumenta el umbral de detección la situación comienza a revertirse: el filtro empeora la detección del correo spam con mas falsos negativos, pero se reduce la cantidad de falsos positivos que detecta.
3. Esta relación entre precisión (cuantos datos de un tipo han sido clasificados correctamente) y exhaustividad (cuantos datos no han sido malinterpretados como el contrario) se puede apreciar en las graficas precision-recall, en la curva de la esquina derecha. Es la región donde se pueden obtener los resultados con el mejor compromiso entre ambas.